<a href="https://colab.research.google.com/github/Thibisay/Proyecto-SIC-cubitUCV/blob/Andreina/Analisis%20de%20datos/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#@title Importaciones
from transformers import pipeline, AutoTokenizer, AutoModelForSeq2SeqLM
import kagglehub, pandas as pd, os
import numpy
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


In [2]:
#@title Si estas desde colab
path = kagglehub.dataset_download("svanoo/myanimelist-dataset")

#Seleccionando el archivo con los animes
anime_dataset = pd.read_csv(f"{path}/anime.csv", sep = '\t',on_bad_lines='skip')
#anime_emotion = pd.read_csv("./anime_emotion.csv")

100%|██████████| 1.78G/1.78G [00:17<00:00, 112MB/s]

Extracting files...


In [ ]:
#@title Si no estas en colab

anime_dataset = pd.read_csv("./anime.csv", sep = '\t',on_bad_lines='skip')
anime_emotion = pd.read_csv("./anime_emotion.csv")

In [3]:
#@title importacion de archivos y formateo de columnas

anime_dataset.columns = anime_dataset.columns.str.lower()
anime_dataset.columns = anime_dataset.columns.str.replace(' ', '_')

In [4]:
anime_dataset = anime_dataset[anime_dataset['anime_id'] != 44303] # anime que cumple con los 200 caracteres en synopsis pero no contiene datos utiles para su estudio
anime_dataset_filt = pd.DataFrame(anime_dataset[ anime_dataset["synopsis"].str.len() >= 200 ])
anime_dataset_filt = anime_dataset_filt.sort_values(by='popularity_rank')
anime_dataset_filt = anime_dataset_filt.dropna(subset=['score'])
anime_dataset_filt.index = range(0,len(anime_dataset_filt))


In [5]:
#@title normalizacion de los generos
anime_dataset_filt['genres'] = anime_dataset_filt['genres'].apply(lambda x: x.split('|'))
data_generos = pd.DataFrame(list(set(elemento for sublista in anime_dataset_filt['genres'] for elemento in sublista)), columns=['genres_name'])
anime_genres_relation = anime_dataset_filt.explode('genres')[["anime_id", "genres"]] # expande anime_dataset_filt para dividir las listas en filas individuales

data_generos = anime_genres_relation.groupby('genres').size().reset_index(name='cantidad') # Agrupamos por el género y contamos la cantidad de animes en cada uno
data_generos.columns = ['genres_name', 'cantidad']

data_generos

,genres_name,cantidad
0,Action,2436
1,Adventure,1610
2,Avant Garde,95
3,Award Winning,3
4,Boys Love,109
5,Cars,48
6,Comedy,2860
7,Demons,317
8,Drama,1601
9,Ecchi,475


In [6]:
#@title Eliminando columnas

col_delete_list = ["genres","score_count", "score_rank","type", "status", "num_episodes", "start_date",
                   "end_date", "season", "studios", "clubs", "score_01_count",
                   "score_02_count", "score_03_count", "score_04_count", "score_05_count",
                   "score_06_count", "score_07_count", "score_08_count", "score_09_count",
                   "score_10_count", "num_episodes", "watching_count",
                   "completed_count","on_hold_count","dropped_count"]

filter_columns = list(filter(lambda x: True if x not in col_delete_list else False, list(anime_dataset_filt.columns)))

anime_dataset_filt = anime_dataset_filt[filter_columns]

anime_dataset_filt['longitud_synopsis_caracteres'] = anime_dataset_filt['synopsis'].apply(len)


<ipython-input-6-50a437ca5fcf>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  anime_dataset_filt['longitud_synopsis_caracteres'] = anime_dataset_filt['synopsis'].apply(len)


In [ ]:
#@title Analisis de correlacion entre longitud de sinopsis y la puntuacion

correlacion_longSynop_populRank = anime_dataset_filt.select_dtypes(include=[float,int]).corr()

correlacion_longSynop_populRank

In [ ]:
#@title Analisis de correlacion entre longitud de sinopsis y la puntuacion
correlacion_longSynop_score = anime_dataset_filt[['longitud_synopsis_caracteres', 'score']].corr()


# plt.scatter(anime_dataset_filt['longitud_synopsis_caracteres'], anime_dataset_filt['score'], color='blue')
# plt.title('Relación entre Longitud de la Sinopsis y Rango de Popularidad')
# plt.xlabel('Longitud de la Sinopsis')
# plt.ylabel('La puntuación')
# plt.show()
anime_dataset_filt[['longitud_synopsis_caracteres', 'score']]
anime_dataset_filt.groupby('score').agg({'score':'count'})
correlacion_longSynop_score
# anime_dataset_filt[["genres", "anime_id"]]

In [23]:
#@title Modelo de IA, no ejecutar si estas en vsc


# Cargar un pipeline de clasificación de emociones
classifier = pipeline("text-classification", model="j-hartmann/emotion-english-distilroberta-base", tokenizer="j-hartmann/emotion-english-distilroberta-base" , max_length=512, return_all_scores=True)
# Función para analizar emociones de un texto dado
def detectar_emociones(texto):
    resultados = classifier(texto)#aja yo estoy aqui
    # Procesar y mostrar los resultados de manera ordenada
    emociones = {resultado['label']: resultado['score'] for resultado in resultados[0]}
    emociones_ordenadas = sorted(emociones.items(), key=lambda item: item[1], reverse=True)

    return emociones_ordenadas


# Ejemplo de uso
texto = "I'm going under and this time I fear there's no one to save me"

emociones_detectadas = detectar_emociones(texto)

def print_results(emociones_detectadas):
  print("Emociones detectadas:")
  for emocion, puntaje in emociones_detectadas:
      print(f"{emocion}: {puntaje:.2f}")



Device set to use cuda:0
/usr/local/lib/python3.11/dist-packages/transformers/pipelines/text_classification.py:106: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


In [ ]:
#@title Otro modelo no ejecutar


# Cargar el modelo y el tokenizador T5 ajustado para emociones
modelo_id = "mrm8488/t5-base-finetuned-emotion"
tokenizer = AutoTokenizer.from_pretrained(modelo_id)
modelo = AutoModelForSeq2SeqLM.from_pretrained(modelo_id)

# Función para detectar la emoción principal en un texto dado
def detectar_emocion_t5(texto):
    # Preprocesar el texto para el modelo T5
    entradas = tokenizer("emotion: " + texto, return_tensors="pt", padding=True)

    # Generar la emoción usando el modelo
    salida = modelo.generate(**entradas)

    # Decodificar la emoción generada
    emocion = tokenizer.decode(salida[0], skip_special_tokens=True)
    return emocion



In [8]:
from google.colab import drive
drive.mount('/content/drive')
ruta="/content/drive/MyDrive/COLAB" # ruta del drive de wina

Mounted at /content/drive


In [ ]:
#@title Clasificacion por emociones

try:
    emociones_animes = pd.read_csv('anime_emotion.csv')
except FileNotFoundError:
    # Si el archivo no existe, crear un DataFrame vacío con las columnas necesarias
    emociones_animes = pd.DataFrame(columns=['anime_id', 'fear', 'sadness', 'neutral', 'joy', 'anger', 'surprise', 'disgust'])

anime_dataset_filt[["title", "synopsis"]].head()
synopsis = anime_dataset_filt["synopsis"].head()
i = 0
#emociones_animes = pd.DataFrame(columns=['anime_id', 'fear', 'sadness', 'neutral', 'joy', 'anger', 'surprise', 'disgust'])

# Iterar sobre las filas del dataset filtrado
for element in anime_dataset_filt[["anime_id", "title", "synopsis"]][len(emociones_animes):].itertuples():
  # Detectar la emoción en el synopsis
  emociones = detectar_emociones(element.synopsis)

  # Convertir la lista de tuplas en un diccionario
  emociones_dict = {emocion: prob for emocion, prob in emociones}

  # Agregar el anime_id al diccionario
  emociones_dict['anime_id'] = element.anime_id

  # Convertir el diccionario en un DataFrame y concatenarlo con el DataFrame principal
  emociones_animes = pd.concat([emociones_animes, pd.DataFrame([emociones_dict])], ignore_index=True)
  i+=1

  print(i,element.title)
  # if i ==10:
  #   break
emociones_animes.to_csv("anime_emotion.csv", index=False)

1 Godzilla 3: Hoshi wo Kuu Mono
2 Nyoro-n Churuya-san
3 Macross F Movie 1: Itsuwari no Utahime
4 Toshi Densetsu Series
5 Blue Drop: Tenshi-tachi no Gikyoku
6 Aishen Qiaokeli-ing...II
7 Wo de Tian Jie Nu You
8 Onegai☆Twins: Natsu wa Owaranai
9 Kuroko no Basket Movie 2: Winter Cup - Namida no Saki e
10 Lodoss-tou Senki: Eiyuu Kishi Den
11 Skull Man
12 Chikyuu Shoujo Arjuna
13 Kurau Phantom Memory
14 Kyou kara Ore wa!!
15 Sex Pistols
16 Rescue Me!
17 Kirarin☆Revolution
18 Dragon Ball Specials
19 Lostorage Conflated WIXOSS
20 Tenchi Muyou! GXP
21 Choboraunyopomi Gekijou Ai Mai Mii
22 Blame!
23 The Reflection
24 Katte ni Kaizou
25 D.C.III: Da Capo III
26 One Piece: Oounabara ni Hirake! Dekkai Dekkai Chichi no Yume!
27 Minami-ke Betsubara
28 Tian Guan Ci Fu Special
29 Mezzo Forte
30 Giovanni no Shima
31 Papa datte, Shitai
32 Ashita no Nadja
33 Venus Versus Virus
34 Cat Shit One
35 Amai Choubatsu: Watashi wa Kanshu Senyou Pet
36 Kachou Ouji
37 Kiitarou Shounen no Youkai Enikki
38 Eiken: Eiken

In [ ]:
#@title merge entre los animes y los generos


anime_dataset_filt
group_genres = list(anime_genres_relation.groupby("genres"))
merged_df = pd.merge(anime_dataset_filt, anime_genres_relation, on="anime_id", how="left")
merged_df = pd.merge(merged_df, anime_emotion, on="anime_id", how="left")
#merged_df.corr()
merged_df

In [ ]:
#@title Grafico de score promedio por genero
# Grafico de la media
df = merged_df.groupby("genres")["score"].agg(['mean','median','std', 'count']).sort_values(by="count", ascending=False)
df[['mean']].plot(kind="bar", color="skyblue", edgecolor="black")
plt.title("Media en puntaje por género", fontsize=16)
plt.xlabel("Género", fontsize=12)
plt.ylabel("Puntaje promedio", fontsize=12)
plt.xticks(rotation=90, fontsize=10)
plt.grid(axis="y", linestyle="--", alpha=0.7)

# Mostrar el gráfico
plt.tight_layout()  # Ajustar para evitar recortes
plt.show()

# Grafico de la mediana
df[['median']].plot(kind="bar", color="skyblue", edgecolor="black")
plt.title("Mediana en puntaje por género", fontsize=16)
plt.xlabel("Género", fontsize=12)
plt.ylabel("Puntaje promedio", fontsize=12)
plt.xticks(rotation=90, fontsize=10)
plt.grid(axis="y", linestyle="--", alpha=0.7)
plt.tight_layout()  # Ajustar para evitar recortes
plt.show()

# Grafico de la mediana
df[['std']].plot(kind="bar", color="skyblue", edgecolor="black")
plt.title("Varianza de la puntuación por género", fontsize=16)
plt.xlabel("Género", fontsize=12)
plt.ylabel("Varianza", fontsize=12)
plt.xticks(rotation=90, fontsize=10)
plt.grid(axis="y", linestyle="--", alpha=0.7)
plt.tight_layout()  # Ajustar para evitar recortes
plt.show()



In [ ]:
#@title Correlación entre los géneros y el score
import seaborn as sns

df_encoded = pd.get_dummies(merged_df[['score','genres']], columns=['genres'])
genre_score_corr = df_encoded.corr()['score'].drop('score')
genre_score_corr.sort_values(ascending=False).plot(kind='bar', color='skyblue', edgecolor='black')
plt.title('Correlación entre géneros y puntaje')
plt.xlabel('Género')
plt.ylabel('Correlación')
plt.xticks(rotation=90, fontsize=7)
plt.tight_layout()
plt.show()

plt.figure(figsize=(10, 8))
sns.heatmap(df_encoded.corr(), annot=False, cmap='coolwarm', fmt=".2f")
plt.title('Mapa de calor de correlaciones')
plt.show()

In [ ]:
#@title Analisis de emociones con la puntuación

#@title Grafico de score promedio por genero
# Grafico de la media
df = merged_df.groupby("emotion")["score"].agg(['mean','median','std', 'count']).sort_values(by="count", ascending=False)
df[['mean']].plot(kind="bar", color="skyblue", edgecolor="black", legend=False)
plt.title("Media en puntaje por emoción", fontsize=16)
plt.xlabel("Emoción", fontsize=12)
plt.ylabel("Puntaje promedio", fontsize=12)
plt.xticks(rotation=90, fontsize=10)
plt.grid(axis="y", linestyle="--", alpha=0.7)

# Mostrar el gráfico
plt.tight_layout()  # Ajustar para evitar recortes
plt.show()

# Grafico de la mediana
df[['median']].plot(kind="bar", color="skyblue", edgecolor="black", legend=False)
plt.title("Mediana en puntaje por emoción", fontsize=16)
plt.xlabel("Emoción", fontsize=12)
plt.ylabel("Puntaje promedio", fontsize=12)
plt.xticks(rotation=90, fontsize=10)
plt.grid(axis="y", linestyle="--", alpha=0.7)
plt.tight_layout()  # Ajustar para evitar recortes
plt.show()

# Grafico de la mediana
df[['std']].plot(kind="bar", color="skyblue", edgecolor="black", legend=False)
plt.title("Varianza de la puntuación por emoción", fontsize=16)
plt.xlabel("Emoción", fontsize=12)
plt.ylabel("Varianza", fontsize=12)
plt.xticks(rotation=90, fontsize=10)
plt.grid(axis="y", linestyle="--", alpha=0.7)
plt.tight_layout()  # Ajustar para evitar recortes
plt.show()



In [ ]:
#@title Correlación entre las emociones y el score
import seaborn as sns

df_encoded = pd.get_dummies(merged_df[['score','emotion']], columns=['emotion'])
genre_score_corr = df_encoded.corr()['score'].drop('score')
genre_score_corr.sort_values(ascending=False).plot(kind='bar', color='skyblue', edgecolor='black')
plt.title('Correlación entre emociones y puntaje')
plt.xlabel('Emoción')
plt.ylabel('Correlación')
plt.xticks(rotation=90, fontsize=7)
plt.tight_layout()
plt.show()

plt.figure(figsize=(10, 8))
sns.heatmap(df_encoded.corr(), annot=True, cmap='coolwarm', fmt=".2f")
plt.title('Mapa de calor de correlaciones')
plt.show()

In [ ]:
#@title media,mediana,varianza - popularidad

# Grafico de la media
df = merged_df.groupby("genres")["popularity_rank"].agg(['mean','median','std', 'count']).sort_values(by="count", ascending=False)
df[['mean']].plot(kind="bar", color="skyblue", edgecolor="black")
plt.title("Media en puntaje por popularidad", fontsize=16)
plt.xlabel("popularidad", fontsize=12)
plt.ylabel("Puntaje promedio", fontsize=12)
plt.xticks(rotation=90, fontsize=10)
plt.grid(axis="y", linestyle="--", alpha=0.7)


In [ ]:
#@title generos - favoritos

# Grafico de la media
df = merged_df.groupby("genres")["favorites_count"].agg(['mean','median','std', 'count']).sort_values(by="count", ascending=False)
df[['mean']].plot(kind="bar", color="skyblue", edgecolor="black")
plt.title("Media en puntaje por ranking de favoritos", fontsize=16)
plt.xlabel("favoritos", fontsize=12)
plt.ylabel("Puntaje promedio", fontsize=12)
plt.xticks(rotation=90, fontsize=10)
plt.grid(axis="y", linestyle="--", alpha=0.7)


In [ ]:
#@title emociones - fav


# Grafico de la media
df = merged_df.groupby("emotion")["favorites_count"].agg(['mean','median','std', 'count']).sort_values(by="count", ascending=False)
df[['mean']].plot(kind="bar", color="skyblue", edgecolor="black")
plt.title("Media en puntaje por ranking de favoritos", fontsize=16)
plt.xlabel("favoritos", fontsize=12)
plt.ylabel("Puntaje promedio", fontsize=12)
plt.xticks(rotation=90, fontsize=10)
plt.grid(axis="y", linestyle="--", alpha=0.7)


In [ ]:
#@title emociones - popularidad

# Grafico de la media
df = merged_df.groupby("emotion")["popularity_rank"].agg(['mean','median','std', 'count']).sort_values(by="count", ascending=False)
df[['mean']].plot(kind="bar", color="skyblue", edgecolor="black")
plt.title("Media en puntaje por popularidad", fontsize=16)
plt.xlabel("popularidad", fontsize=12)
plt.ylabel("Puntaje promedio", fontsize=12)
plt.xticks(rotation=90, fontsize=10)
plt.grid(axis="y", linestyle="--", alpha=0.7)

# New Section

In [ ]:
from matplotlib import pyplot as plt
import seaborn as sns
emociones_animes.groupby('emotion').size().plot(kind='barh', color=sns.palettes.mpl_palette('Dark2'))
plt.gca().spines[['top', 'right',]].set_visible(False)
plt.ylabel('Emociones')
plt.xlabel('Cantidad')
plt.title('Cantidad de animes por emoción')
plt.show()

In [ ]:

anime_dataset_filt['popularidad_categoria'] = pd.cut(anime_dataset_filt['popularity_rank'],bins=[0, 2000, 4000, 6000, 8000, anime_dataset_filt['popularity_rank'].max()],labels=['Muy Popular', 'Popular', 'Moderado', 'Bajo', 'Muy Bajo'])


# Función para obtener emociones para cada anime usando id_anime
def obtener_emociones(id_anime):
    # Filtramos emociones_animes para obtener las emociones asociadas a un id_anime
    emociones = emociones_animes[emociones_animes['anime_id'] == id_anime]['emotion'].tolist()
    return emociones

# Crear una lista para almacenar los resultados
emociones_popularidad = []

# Recorrer cada anime en anime_dataset_filt
for _, anime in anime_dataset_filt.iterrows():
    # Obtener las emociones del anime
    emociones = obtener_emociones(anime['anime_id'])
    # Obtener la categoría de popularidad
    categoria_popularidad = anime['popularidad_categoria']

    # Guardar las emociones junto con la categoría de popularidad
    for emocion in emociones:
        emociones_popularidad.append([categoria_popularidad, emocion])

# Convertir los resultados a un DataFrame
df_emociones_popularidad = pd.DataFrame(emociones_popularidad, columns=['popularidad_categoria', 'emociones'])

# Contar las emociones por popularidad
emotion_counts = df_emociones_popularidad.groupby(['popularidad_categoria', 'emociones']).size().unstack(fill_value=0)
categorias_ordenadas = ['Muy Popular', 'Popular', 'Moderado', 'Bajo', 'Muy Bajo']
emotion_counts.index = pd.CategoricalIndex(emotion_counts.index, categories=categorias_ordenadas, ordered=True)

emotion_counts = emotion_counts.sort_index()
emotion_counts

In [ ]:
fig, axes = plt.subplots(1, 5, figsize=(20, 6), sharey=True)

# Crear un gráfico para cada categoría
for i, categoria in enumerate(emotion_counts.index):
    ax = axes[i]
    datos_categoria = emotion_counts.loc[categoria]
    emotion_counts.loc[categoria].plot(kind='bar', ax=axes[i], color='skyblue', edgecolor='black')
    axes[i].set_title(f'Popularidad: {categoria}')
    axes[i].set_xlabel('Emociones')
    axes[i].set_ylabel('Cantidad de Animes')
    axes[i].tick_params(axis='x', rotation=45)

    # Agregar los valores encima de las barras
    for j, valor in enumerate(datos_categoria):
        ax.text(j, valor + 0.5, str(valor), ha='center', va='bottom', fontsize=10)

# Agregar la leyenda para los rangos de puntuación
fig.text(0.5, 1.02, "Cantidad De animes por Categorías de Popularidad y Emociones", ha='center', va='center', fontsize=20)

rango_puntuaciones = [
    "Muy Popular: 1 - 2000 Rango",
    "Popular: 2001 - 4000 Rango",
    "Moderado: 4001 - 6000 Rango",
    "Bajo: 6001 - 8000 Rango",
    "Muy Bajo: 8001 - 10000 Rango"
]
fig.legend(
    rango_puntuaciones,
    loc="upper center",
    ncol=5,
    bbox_to_anchor=(0.5, 1.15),
    fontsize='medium',
    frameon=False
)

# Ajustar el diseño
plt.tight_layout(rect=[0, 0, 1, 0.95])  # Reservar espacio superior para la leyenda
plt.show()

In [ ]:
# Visualización: Gráfico de barras apiladas
emotion_counts.sum(axis=1).plot(kind='bar', figsize=(10, 8), color = 'skyblue' )
plt.title('Cantidad de animes por categoría de Popularidad')
plt.xlabel('Categoría de popularidad')
plt.ylabel('Cantidad de animes')
plt.xticks(rotation=45)
plt.show()

In [ ]:

# Crear las categorías de score en anime_dataset_filt
anime_dataset_filt['score_categoria'] = pd.cut(anime_dataset_filt['score'],
                                               bins=[0, 3, 5, 7, 8, 10],
                                               labels=['Bajo', 'Medio Bajo', 'Medio Alto', 'Alto', 'Excelente'])

# Función para obtener emociones para cada anime usando id_anime
def obtener_emociones(id_anime):
    # Filtramos emociones_animes para obtener las emociones asociadas a un id_anime
    emociones = emociones_animes[emociones_animes['anime_id'] == id_anime]['emotion'].tolist()
    return emociones

# Crear una lista para almacenar los resultados
emociones_score = []

# Recorrer cada anime en anime_dataset_filt
for _, anime in anime_dataset_filt.iterrows():
    # Obtener las emociones del anime
    emociones = obtener_emociones(anime['anime_id'])
    # Obtener la categoría de score
    categoria_score = anime['score_categoria']

    # Guardar las emociones junto con la categoría de score
    for emocion in emociones:
        emociones_score.append([categoria_score, emocion])

# Convertir los resultados a un DataFrame
df_emociones_score = pd.DataFrame(emociones_score, columns=['score_categoria', 'emociones'])

# Contar las emociones por score
emotion_counts_score = df_emociones_score.groupby(['score_categoria', 'emociones']).size().unstack(fill_value=0)

# Asegurar que los índices de score estén en el orden correcto
# Especificar el orden de las categorías de score
categorias_score_ordenadas = ['Bajo', 'Medio Bajo', 'Medio Alto', 'Alto', 'Excelente']
emotion_counts_score.index = pd.CategoricalIndex(emotion_counts_score.index, categories=categorias_score_ordenadas, ordered=True)

# Reorganizar el DataFrame para asegurar que el índice de score esté en el orden correcto
emotion_counts_score = emotion_counts_score.sort_index()
emotion_counts_score

In [ ]:
fig, axes = plt.subplots(1, 5, figsize=(20, 6), sharey=True)

# Crear un gráfico para cada categoría
for i, categoria in enumerate(emotion_counts_score.index):
    ax = axes[i]
    datos_categoria = emotion_counts_score.loc[categoria]

    # Crear el gráfico de barras
    datos_categoria.plot(kind='bar', ax=ax, color='skyblue', edgecolor='black')

    # Títulos y etiquetas
    ax.set_title(f'Categoría: {categoria}')
    ax.set_xlabel('Emociones')
    ax.set_ylabel('Cantidad de Animes')
    ax.tick_params(axis='x', rotation=45)

    # Agregar los valores encima de las barras
    for j, valor in enumerate(datos_categoria):
        ax.text(j, valor + 0.5, str(valor), ha='center', va='bottom', fontsize=10)


# Agregar la leyenda para los rangos de puntuación
fig.text(0.5, 1.02, "Animes por Categorías de Puntuación y Emociones", ha='center', va='center', fontsize=20)

rango_puntuaciones = [
    "Bajo: 0 - 2 puntos",
    "Medio Bajo: 3 - 4 puntos",
    "Medio Alto: 5 - 6 puntos",
    "Alto: 7 - 8 puntos",
    "Excelente: 9 - 10 puntos"
]
fig.legend(

    rango_puntuaciones,
    loc="upper center",
    ncol=5,
    bbox_to_anchor=(0.5, 1.15),
    fontsize='medium',
    frameon=False
)

# Ajustar el diseño
plt.tight_layout(rect=[0, 0, 1, 0.95])  # Reservar espacio superior para la leyenda
plt.show()


In [ ]:
# Crear el gráfico de barras simple
ax = emotion_counts_score.sum(axis=1).plot(kind='bar', color='skyblue',figsize=(10, 6))

# Establecer el título y las etiquetas
plt.title('Cantidad de animes por categoría de Puntos')
plt.xlabel('Categoría de Puntos')
plt.ylabel('Cantidad de Animes')

# Rotar las etiquetas del eje X
plt.xticks(rotation=45)

# Ajustar el diseño para evitar superposiciones
plt.tight_layout()

# Mostrar el gráfico
plt.show()
